In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix

In [3]:
titanic = pd.read_csv('titanic_df.csv')
titanic.head()

,Unnamed: 0,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1


## Using the titanic data, in your classification-exercises repository, create a notebook, model.ipynb where you will do the following:

- What is your baseline prediction? 

- What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). 

- When you make those predictions, what is your accuracy? This is your baseline accuracy.

In [ ]:
titanic.info()

In [ ]:
X = titanic[['pclass', 'fare']]
y = titanic.survived

X_train_and_validate, X_test, y_train_and_validate, y_test = train_test_split(X, y, random_state=123, test_size=.3)
X_train, X_validate, y_train, y_validate = train_test_split(X_train_and_validate, y_train_and_validate, random_state=123, test_size=.2)

print("train: ", X_train.shape, ", validate: ", X_validate.shape, ", test: ", X_test.shape)
print("train: ", y_train.shape, ", validate: ", y_validate.shape, ", test: ", y_test.shape)

In [ ]:
y_train.value_counts()

In [ ]:
baseline = y_train.mode()


matches_baseline_prediction = (y_train == 0)

baseline_accuracy = matches_baseline_prediction.mean()
print(f"Baseline accuracy: {round(baseline_accuracy, 2)}")

- Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

In [ ]:
tree = DecisionTreeClassifier(max_depth = 20)

In [ ]:
tree.fit(X_train, y_train)


In [ ]:
print(export_text(tree, feature_names=X_train.columns.tolist()))

In [ ]:
y_predictions = tree.predict(X_train)
y_predictions

- Evaluate your in-sample results using the model score, confusion matrix, and classification report.

In [ ]:
#confusematrix
pd.DataFrame(confusion_matrix(y_train, y_predictions))

In [ ]:
pd.crosstab(y_train, y_predictions)

In [ ]:
print(classification_report(y_train, y_predictions))

Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [ ]:
print(classification_report(y_train, y_predictions))

Run through steps 2-4 using a different max_depth value.

In [ ]:
tree1 = DecisionTreeClassifier(max_depth = 1)
tree1

In [ ]:
tree1 = tree.fit(X_train, y_train)
tree1

In [ ]:
#i decided to jus timput diffrent values for max depth and re run the lines of code to see the diffrence 

Which model performs better on your in-sample data?

In [ ]:
#looks like a max depth of 20 looks very accurate 

Which model performs best on your out-of-sample data, the validate set?

In [ ]:
tree.score(X_validate, y_validate) 

In [ ]:
y_validate.value_counts(normalize=True)

In [ ]:
#better accurace to guess not survived in the data

 ## Work through these same exercises using the Telco dataset.

- What is your baseline prediction? What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). 
- When you make those predictions, what is your accuracy? This is your baseline accuracy.

In [ ]:
telco = pd.read_csv('telco.csv')
telco.head()

In [ ]:
telco.isna().sum()

## Random forest Excercizes 

- Fit the Random Forest classifier to your training sample and transform (i.e. make predictions on the training sample) setting the random_state accordingly and setting min_samples_leaf = 1 and max_depth = 10.

In [4]:
titanic = pd.read_csv('titanic_df.csv')
titanic.head()

,Unnamed: 0,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1


In [5]:
titanic = titanic.drop(columns=["class", "embarked","deck"])
titanic

,Unnamed: 0,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embark_town,alone
0,0,0,0,3,male,22.0,1,0,7.2500,Southampton,0
1,1,1,1,1,female,38.0,1,0,71.2833,Cherbourg,0
2,2,2,1,3,female,26.0,0,0,7.9250,Southampton,1
3,3,3,1,1,female,35.0,1,0,53.1000,Southampton,0
4,4,4,0,3,male,35.0,0,0,8.0500,Southampton,1
...,...,...,...,...,...,...,...,...,...,...,...
886,886,886,0,2,male,27.0,0,0,13.0000,Southampton,1
887,887,887,1,1,female,19.0,0,0,30.0000,Southampton,1
888,888,888,0,3,female,NaN,1,2,23.4500,Southampton,0
889,889,889,1,1,male,26.0,0,0,30.0000,Cherbourg,1


In [8]:
titanic.age = titanic.age.fillna(value=titanic.age.median())

titanic

,Unnamed: 0,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embark_town,alone
0,0,0,0,3,male,22.0,1,0,7.2500,Southampton,0
1,1,1,1,1,female,38.0,1,0,71.2833,Cherbourg,0
2,2,2,1,3,female,26.0,0,0,7.9250,Southampton,1
3,3,3,1,1,female,35.0,1,0,53.1000,Southampton,0
4,4,4,0,3,male,35.0,0,0,8.0500,Southampton,1
...,...,...,...,...,...,...,...,...,...,...,...
886,886,886,0,2,male,27.0,0,0,13.0000,Southampton,1
887,887,887,1,1,female,19.0,0,0,30.0000,Southampton,1
888,888,888,0,3,female,28.0,1,2,23.4500,Southampton,0
889,889,889,1,1,male,26.0,0,0,30.0000,Cherbourg,1


In [9]:
dummy_df = pd.get_dummies(titanic[['sex','embark_town']], dummy_na=False, drop_first=[True, True])


titanic = titanic.drop(columns=["sex", "embark_town"])

titanic = pd.concat([titanic, dummy_df], axis=1)
titanic.head()

,Unnamed: 0,passenger_id,survived,pclass,age,sibsp,parch,fare,alone,sex_male,embark_town_Queenstown,embark_town_Southampton
0,0,0,0,3,22.0,1,0,7.2500,0,1,0,1
1,1,1,1,1,38.0,1,0,71.2833,0,0,0,0
2,2,2,1,3,26.0,0,0,7.9250,1,0,0,1
3,3,3,1,1,35.0,1,0,53.1000,0,0,0,1
4,4,4,0,3,35.0,0,0,8.0500,1,1,0,1


In [10]:
def train_validate_test_split(df, target, seed=123):
    '''
    This function takes in a dataframe, the name of the target variable
    (for stratification purposes), and an integer for a setting a seed
    and splits the data into train, validate and test. 
    Test is 20% of the original dataset, validate is .30*.80= 24% of the 
    original dataset, and train is .70*.80= 56% of the original dataset. 
    The function returns, in this order, train, validate and test dataframes. 
    '''
    train_validate, test = train_test_split(titanic, test_size=0.2, 
                                            random_state=seed, 
                                            stratify=df[target])
    train, validate = train_test_split(train_validate, test_size=0.3, 
                                       random_state=seed,
                                       stratify=train_validate[target])
    return train, validate, test

In [11]:
train, validate, test = train_validate_test_split(titanic, target='survived', seed=123)

# Explore your data here. 

# create X & y version of train, where y is a series with just the target variable and X are all the features. 

X_train = train.drop(columns=['survived'])
y_train = train.survived

X_validate = validate.drop(columns=['survived'])
y_validate = validate.survived

X_test = test.drop(columns=['survived'])
y_test = test.survived


In [12]:
X_train

,Unnamed: 0,passenger_id,pclass,age,sibsp,parch,fare,alone,sex_male,embark_town_Queenstown,embark_town_Southampton
583,583,583,1,36.0,0,0,40.1250,1,1,0,0
165,165,165,3,9.0,0,2,20.5250,0,1,0,1
50,50,50,3,7.0,4,1,39.6875,0,1,0,1
259,259,259,2,50.0,0,1,26.0000,0,0,0,1
306,306,306,1,28.0,0,0,110.8833,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
313,313,313,3,28.0,0,0,7.8958,1,1,0,1
636,636,636,3,32.0,0,0,7.9250,1,1,0,1
222,222,222,3,51.0,0,0,8.0500,1,1,0,1
485,485,485,3,28.0,3,1,25.4667,0,0,0,1


In [13]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
rf = RandomForestClassifier(max_depth=3, 
                            random_state=123)
rf

RandomForestClassifier(max_depth=3, random_state=123)

In [16]:
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=3, random_state=123)

In [17]:
y_pred = rf.predict(X_train)
y_pred

array([0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

- Evaluate your results using the model score, confusion matrix, and classification report.

In [18]:
print(confusion_matrix(y_train, y_pred))

[[294  13]
 [ 66 125]]


In [19]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.96      0.88       307
           1       0.91      0.65      0.76       191

    accuracy                           0.84       498
   macro avg       0.86      0.81      0.82       498
weighted avg       0.85      0.84      0.83       498



- Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.



- Run through steps increasing your min_samples_leaf and decreasing your max_depth.

- What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?